In [23]:
import pandas as pd
from Bio import SeqIO

# convert fasta file to dataframe
def convert_fasta_to_df(fasta_file):
    names = []
    descriptions = []
    sequences = []
    
    for record in SeqIO.parse(fasta_file, 'fasta'):
        names.append(record.id)
        descriptions.append(record.description)
        sequences.append(str(record.seq))
    
    dataframe = pd.DataFrame({
        'name': names,
        'description': descriptions,
        'sequence': sequences
    })
    return dataframe

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# convert a dataframe with columns['name', 'sequence'] to fasta file
# specify dataframe and output file path/name as parameters
def convert_df_to_fasta(dataframe, output_file):
    records = [SeqRecord(Seq(row['sequence']), id=row['name'], description='') for index, row in dataframe.iterrows()]
    
    with open(output_file, "w") as fasta_file:
        SeqIO.write(records, fasta_file, "fasta")

def check_chop_signalp(signalp_gff, df_to_chop, id_column, sequence_column):
    # read the signalp output file
    columns=['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attribute']
    signal = pd.read_csv(signalp_gff, sep='\t', comment='#', names=columns)
    signal = signal.loc[:,['seqid', 'start', 'end', 'score']]
    signal['seqid'] = signal['seqid'].str.split().str[0]
    # chop the signal peptide off and update the length information
    df_chopped = pd.merge(df_to_chop, signal, left_on=id_column, right_on='seqid', how='left')

    # fill the default start with 0 for those without signalp
    df_chopped['start'].fillna(0, inplace=True)

    # chop the signalp off from original sequences
    if df_chopped['start'].isin([0,1]).all():
        df_chopped['end'] = df_chopped['end'].astype('Int64')
        df_chopped['end'].fillna(0, inplace=True)
        df_chopped.loc[df_chopped['seqid'].notnull(), sequence_column+'_wo_sp'] = df_chopped.apply(lambda row: row[sequence_column][row['end']:], axis=1)
    else:
        print('Not all starts at pos 1')
        
    # update the sequence length to 'length_new'
    df_chopped.loc[df_chopped['seqid'].notnull(),'length_wo_sp'] = df_chopped['length'] - df_chopped['end']
    df_chopped.drop(columns=['seqid', 'start', 'end', 'score'], inplace=True)
    
    return df_chopped

In [ ]:
# convert orf fasta file from emboss getorf program to dataframe
# get orf by $getorf -sequence sleb_pr5_fold2_noref_nooverlap_seq.fasta -find 1 -minsize 15 -outseq test.txt
# /rugpfs/fs0/zhao_lab/scratch/cli02/infection_pr/analysis/assembly/merged/orf
orf = convert_fasta_to_df('orf/orf_sleb.txt')

# add some columns
orf['transcript_id'] = orf['name'].str.split('_').str[0]
orf['gene_id'] = orf['transcript_id'].apply(lambda x: x.rsplit('.', 1)[0])
orf['negative_strand'] = orf['description'].str.contains('REVERSE SENSE')
orf['length'] = orf['sequence'].str.len()

In [241]:
orf

,name,description,sequence,transcript_id,gene_id,negative_strand,length
0,MSTRG.406.1_1,MSTRG.406.1_1 [16 - 42],MYKHTYIYT,MSTRG.406.1,MSTRG.406,False,9
1,MSTRG.406.1_2,MSTRG.406.1_2 [68 - 184],MCPICFTTKVLDAGEFLRLNCFEQSVLEAKRMQTAETSS,MSTRG.406.1,MSTRG.406,False,39
2,MSTRG.406.1_3,MSTRG.406.1_3 [314 - 361],MPRRREQHTCALPTCQ,MSTRG.406.1,MSTRG.406,False,16
3,MSTRG.406.1_4,MSTRG.406.1_4 [355 - 423],MPVRVILVYYALWSQNWLAGITM,MSTRG.406.1,MSTRG.406,False,23
4,MSTRG.406.1_5,MSTRG.406.1_5 [520 - 585],MLYRRARYKILVDYLLISFIGH,MSTRG.406.1,MSTRG.406,False,22
...,...,...,...,...,...,...,...
518,MSTRG.13746.1_181,MSTRG.13746.1_181 [352 - 278] (REVERSE SENSE),MKFFLNNYFKIFFYFLVRHGSLFSR,MSTRG.13746.1,MSTRG.13746,True,25
519,MSTRG.13746.1_182,MSTRG.13746.1_182 [297 - 241] (REVERSE SENSE),MVLSFQDDSRVTEVVTLLM,MSTRG.13746.1,MSTRG.13746,True,19
520,MSTRG.13746.1_183,MSTRG.13746.1_183 [213 - 184] (REVERSE SENSE),MYLYILFGRI,MSTRG.13746.1,MSTRG.13746,True,10
521,MSTRG.13746.1_184,MSTRG.13746.1_184 [238 - 140] (REVERSE SENSE),MISEGCNMYVFIYSFWSDLIAKDTKLMGKSFKS,MSTRG.13746.1,MSTRG.13746,True,33


In [ ]:
# check and chop signal peptides from signalp6 results
# $ signalp6 --fastafile orf_sleb.txt --output_dir signalp_sleb --organism eukarya --mode fast 
# rugpfs/fs0/zhao_lab/scratch/cli02/infection_pr/analysis/assembly/merged/orf/signalp_sleb
orf = check_chop_signalp('orf/output.gff3', orf, 'name', 'sequence')

orf['signal_peptide'] = ~orf['length_new'].isnull()

In [245]:
orf.head(15)

,name,description,sequence,transcript_id,gene_id,negative_strand,length,sequence_new,length_new,signal_peptide
0,MSTRG.406.1_1,MSTRG.406.1_1 [16 - 42],MYKHTYIYT,MSTRG.406.1,MSTRG.406,False,9,NaN,<NA>,False
1,MSTRG.406.1_2,MSTRG.406.1_2 [68 - 184],MCPICFTTKVLDAGEFLRLNCFEQSVLEAKRMQTAETSS,MSTRG.406.1,MSTRG.406,False,39,NaN,<NA>,False
2,MSTRG.406.1_3,MSTRG.406.1_3 [314 - 361],MPRRREQHTCALPTCQ,MSTRG.406.1,MSTRG.406,False,16,NaN,<NA>,False
3,MSTRG.406.1_4,MSTRG.406.1_4 [355 - 423],MPVRVILVYYALWSQNWLAGITM,MSTRG.406.1,MSTRG.406,False,23,NaN,<NA>,False
4,MSTRG.406.1_5,MSTRG.406.1_5 [520 - 585],MLYRRARYKILVDYLLISFIGH,MSTRG.406.1,MSTRG.406,False,22,NaN,<NA>,False
5,MSTRG.406.1_6,MSTRG.406.1_6 [661 - 717],MQILTSMPLFWTFSTARLC,MSTRG.406.1,MSTRG.406,False,19,NaN,<NA>,False
6,MSTRG.406.1_7,MSTRG.406.1_7 [730 - 756],MLEIDPQLG,MSTRG.406.1,MSTRG.406,False,9,NaN,<NA>,False
7,MSTRG.406.1_8,MSTRG.406.1_8 [928 - 1122],MEPNSNLSVNILNSPRPHLYRNCRLIWPYCCLTTMCRFLPPGHYFN...,MSTRG.406.1,MSTRG.406,False,65,NaN,<NA>,False
8,MSTRG.406.1_9,MSTRG.406.1_9 [1115 - 1207],MEHKTVHGGISDRRHLHAARRSAALWSGTKG,MSTRG.406.1,MSTRG.406,False,31,NaN,<NA>,False
9,MSTRG.406.1_10,MSTRG.406.1_10 [1318 - 1371],MPCLRLIYWRIRTYVHLS,MSTRG.406.1,MSTRG.406,False,18,NaN,<NA>,False


In [246]:
# load the information dataframe containing the reference strand info in gtf
info = pd.read_csv('sleb_pr5_fold2_noref_nooverlap.csv')

In [248]:
# add the info the orf dataframe
orf = orf.merge(info[['Gene ID', 'Strand']], how='left', left_on='gene_id', right_on='Gene ID')
orf.drop(columns=['Gene ID'], inplace=True)

# remove the orfs that are on the wrong strand
orf = orf[~((orf['negative_strand']==False) & (orf['Strand']=='-'))]
orf = orf[~((orf['negative_strand']==True) & (orf['Strand']=='+'))]

In [249]:
orf

,name,description,sequence,transcript_id,gene_id,negative_strand,length,sequence_new,length_new,signal_peptide,Strand
0,MSTRG.406.1_1,MSTRG.406.1_1 [16 - 42],MYKHTYIYT,MSTRG.406.1,MSTRG.406,False,9,NaN,<NA>,False,+
1,MSTRG.406.1_2,MSTRG.406.1_2 [68 - 184],MCPICFTTKVLDAGEFLRLNCFEQSVLEAKRMQTAETSS,MSTRG.406.1,MSTRG.406,False,39,NaN,<NA>,False,+
2,MSTRG.406.1_3,MSTRG.406.1_3 [314 - 361],MPRRREQHTCALPTCQ,MSTRG.406.1,MSTRG.406,False,16,NaN,<NA>,False,+
3,MSTRG.406.1_4,MSTRG.406.1_4 [355 - 423],MPVRVILVYYALWSQNWLAGITM,MSTRG.406.1,MSTRG.406,False,23,NaN,<NA>,False,+
4,MSTRG.406.1_5,MSTRG.406.1_5 [520 - 585],MLYRRARYKILVDYLLISFIGH,MSTRG.406.1,MSTRG.406,False,22,NaN,<NA>,False,+
...,...,...,...,...,...,...,...,...,...,...,...
518,MSTRG.13746.1_181,MSTRG.13746.1_181 [352 - 278] (REVERSE SENSE),MKFFLNNYFKIFFYFLVRHGSLFSR,MSTRG.13746.1,MSTRG.13746,True,25,NaN,<NA>,False,-
519,MSTRG.13746.1_182,MSTRG.13746.1_182 [297 - 241] (REVERSE SENSE),MVLSFQDDSRVTEVVTLLM,MSTRG.13746.1,MSTRG.13746,True,19,NaN,<NA>,False,-
520,MSTRG.13746.1_183,MSTRG.13746.1_183 [213 - 184] (REVERSE SENSE),MYLYILFGRI,MSTRG.13746.1,MSTRG.13746,True,10,NaN,<NA>,False,-
521,MSTRG.13746.1_184,MSTRG.13746.1_184 [238 - 140] (REVERSE SENSE),MISEGCNMYVFIYSFWSDLIAKDTKLMGKSFKS,MSTRG.13746.1,MSTRG.13746,True,33,NaN,<NA>,False,-


In [258]:
# group dataframe for each gene id and each strand
orf_groups = orf.groupby(['gene_id', 'negative_strand'])

# get the longest orf in each group
longest_orf = orf_groups['length'].apply(lambda x: max(x))

# Reset the index of longest_sequence to match it with the original DataFrame
longest_orf = longest_orf.reset_index()

# Merge the longest sequences with the original DataFrame
orf_longest_each_strand = pd.merge(orf, longest_orf, on=['gene_id', 'negative_strand', 'length'], how='inner')

In [261]:
# extract candidates with signal peptide
candidate = orf_longest_each_strand[orf_longest_each_strand['signal_peptide']]

In [268]:
candidate

,name,description,sequence,transcript_id,gene_id,negative_strand,length,sequence_new,length_new,signal_peptide,Strand
0,MSTRG.406.1_13,MSTRG.406.1_13 [240 - 1853],MHIEMGFGTMPGRIVLFCLLLISNICRGEGSNILVLFPHASESHFG...,MSTRG.406.1,MSTRG.406,False,538,EGSNILVLFPHASESHFGVLRTLVTELASRDHNVTVYSTHGLGETL...,510,True,+
1,MSTRG.1588.1_1,MSTRG.1588.1_1 [58 - 210],MFFNIRTLLLPLLLLGLALTGTVTEASLQRGRVFDTRPSPYNPNPP...,MSTRG.1588.1,MSTRG.1588,False,51,SLQRGRVFDTRPSPYNPNPPRPGPF,25,True,.
4,MSTRG.2269.1_4,MSTRG.2269.1_4 [249 - 76] (REVERSE SENSE),MRTLSLVYALLMLCFLGFPNLGRAIPFVDRRGGSWGGGQTMRPIPT...,MSTRG.2269.1,MSTRG.2269,True,58,IPFVDRRGGSWGGGQTMRPIPTTPPFNPHASRRF,34,True,.
13,MSTRG.4616.1_6,MSTRG.4616.1_6 [421 - 107] (REVERSE SENSE),MALKYTLFFLACMLAYVAAEQYEFDGNKVRPLGGNGLPEGGRKVEL...,MSTRG.4616.1,MSTRG.4616,True,105,EQYEFDGNKVRPLGGNGLPEGGRKVELKYTDDPAQGGRQSSVRFDQ...,86,True,.
14,MSTRG.6046.1_4,MSTRG.6046.1_4 [195 - 329],MMGQVFMHFALLCCCAATAAQDQICAPIRLASGHGNGQRQRPRLV,MSTRG.6046.1,MSTRG.6046,False,45,QDQICAPIRLASGHGNGQRQRPRLV,25,True,.
19,MSTRG.7930.1_10,MSTRG.7930.1_10 [580 - 398] (REVERSE SENSE),MQFNALCFLLLSLCIVCALNSAVAAVCSGHGFDSRNGNLNLDINNL...,MSTRG.7930.1,MSTRG.7930,True,61,AVCSGHGFDSRNGNLNLDINNLDSLFRCVEQQHRNRG,37,True,.


In [ ]:
# save to csv and fasta files
candidate.to_csv('orf/candidate_orf_sleb.csv')
convert_df_to_fasta(candidate, 'orf/candidate_orf_sleb.fasta')

In [21]:
# standardized way to get orf, add signalp info, add strand info, and get candidates
# prepare orf fasta file as 'orf/orf_'+species+'.txt', info file species+'_pr5_fold2_noref_nooverlap.csv',/
# signalp result gff file 'orf/output.gff3'
# function would output candidate info to file 'orf/candidate_orf_'+species+'.csv' and sequence to 'orf/candidate_orf_'+species+'.fasta'
# also a transcript with signalp only version would be output as well

def process_orf(species): # species name should be hard-encoded species name as 'sleb' as the parameter
    # convert orf fasta file from emboss getorf program to dataframe
    orf = convert_fasta_to_df('orf/orf_'+species+'.txt')
    
    # add some columns
    orf['transcript_id'] = orf['name'].str.split('_').str[0]
    orf['gene_id'] = orf['transcript_id'].apply(lambda x: x.rsplit('.', 1)[0])
    orf['negative_strand'] = orf['description'].str.contains('REVERSE SENSE')
    orf['length'] = orf['sequence'].str.len()
    
    # check and chop signal peptides from signalp6 results
    orf = check_chop_signalp('orf/'+species+'_output.gff3', orf, 'name', 'sequence')
    orf['signal_peptide'] = ~orf['length_wo_sp'].isnull()
    
    # load the information dataframe containing the reference strand info in gtf
    info = pd.read_csv(species+'_ef5_fold2_noref_nooverlap.csv')
    info.drop(columns=['Unnamed: 0', 'Gene Name', '%overlap'], inplace=True)
    
    # add the info the orf dataframe
    orf = orf.merge(info[['Gene ID', 'Strand']], how='left', left_on='gene_id', right_on='Gene ID')
    orf.drop(columns=['Gene ID'], inplace=True)
    
    # remove the orfs that are on the wrong strand
    orf = orf[~((orf['negative_strand']==False) & (orf['Strand']=='-'))]
    orf = orf[~((orf['negative_strand']==True) & (orf['Strand']=='+'))]
    
    # group dataframe for each gene id and each strand
    orf_groups = orf.groupby(['gene_id', 'negative_strand'])
    
    # get the longest orf in each group
    longest_orf = orf_groups['length'].apply(lambda x: max(x))
    
    # Reset the index of longest_sequence to match it with the original DataFrame
    longest_orf = longest_orf.reset_index()
    
    # Merge the longest sequences with the original DataFrame
    orf_longest_each_strand = pd.merge(orf, longest_orf, on=['gene_id', 'negative_strand', 'length'], how='inner')

    candidate = orf_longest_each_strand
    # extract candidates with signal peptide
    candidate_sigp = orf_longest_each_strand[orf_longest_each_strand['signal_peptide']]

    info.drop(columns=['Strand'], inplace=True)
    combined = candidate.merge(info, how='left', left_on='gene_id', right_on='Gene ID')
    combined_sigp = candidate_sigp.merge(info, how='left', left_on='gene_id', right_on='Gene ID')
    
    # save to csv and fasta files
    candidate.to_csv('orf/candidate_orf_'+species+'.csv')
    candidate_sigp.to_csv('orf/candidate_orf_sigp_'+species+'.csv')
    convert_df_to_fasta(candidate, 'orf/candidate_orf_'+species+'.fasta')
    convert_df_to_fasta(candidate_sigp, 'orf/candidate_orf_sigp_'+species+'.fasta')
    combined.to_csv('orf/candidate_orf_combined_'+species+'.csv')
    combined_sigp.to_csv('orf/candidate_orf_combined_sigp_'+species+'.csv')

    print(f'Finished processing orf for {species}.')

In [22]:
species_list = ['dsim', 'dana', 'dvir', 'sleb']
for species in species_list:
    process_orf(species)

Finished processing orf for dsim.
Finished processing orf for dana.
Finished processing orf for dvir.
Finished processing orf for sleb.
